<a href="https://colab.research.google.com/github/rlgomeszz/api-erik/blob/main/pp2AV4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install -q tensorflow==2.12.0
!pip install -q kaggle
!pip install -q pillow
!pip install -q scikit-learn
!pip install -q matplotlib seaborn


ERROR: Could not find a version that satisfies the requirement tensorflow==2.12.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.12.0


In [ ]:
from google.colab import files
files.upload()


In [ ]:
!unzip -q Classroom_Monitoring_Dataset.zip -d data


In [ ]:
import os

for root, dirs, files in os.walk("data"):
    print(root, len(files))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224,224)
BATCH = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

test_gen = ImageDataGenerator(rescale=1./255)

train = train_gen.flow_from_directory(
    "data/train",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical",
    subset="training"
)

val = train_gen.flow_from_directory(
    "data/train",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical",
    subset="validation"
)

test = test_gen.flow_from_directory(
    "data/test",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical",
    shuffle=False
)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

num_classes = train.num_classes

cnn = models.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

cnn.summary()


In [ ]:
cnn.fit(
    train,
    epochs=8,
    validation_data=val
)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

base = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.4)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base.input, outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
history = model.fit(
    train,
    epochs=10,
    validation_data=val
)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def avaliar(modelo, gerador):
    y_true = np.argmax(gerador.classes.reshape(-1,1), axis=1)
    preds = modelo.predict(gerador)
    y_pred = np.argmax(preds, axis=1)
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, average="macro"),
        "recall": recall_score(y_true, y_pred, average="macro"),
        "f1": f1_score(y_true, y_pred, average="macro")
    }

print("CNN:", avaliar(cnn, test))
print("MobileNetV2:", avaliar(model, test))


In [ ]:
model.save("classroom_monitoring_model.h5")


In [ ]:
import pandas as pd

resultados = pd.DataFrame([
    {"modelo":"CNN", **avaliar(cnn, test)},
    {"modelo":"MobileNetV2", **avaliar(model, test)}
])

resultados


In [ ]:
resultados.to_csv("resultados_classroom.csv", index=False)


Aplicação streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import sqlite3
from datetime import datetime


MODEL_PATH = "classroom_monitoring_model.h5"
IMG_SIZE = (224, 224)


CLASSES = ["attentive", "inattentive", "sleeping"]


@st.cache_resource
def load_model():
    return tf.keras.models.load_model(MODEL_PATH)

model = load_model()


def init_db():
    conn = sqlite3.connect("interactions.db", check_same_thread=False)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS interactions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            prediction TEXT,
            confidence REAL
        )
    """)
    conn.commit()
    return conn

conn = init_db()

def save_interaction(prediction, confidence):
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO interactions (timestamp, prediction, confidence) VALUES (?, ?, ?)",
        (datetime.now().isoformat(), prediction, confidence)
    )
    conn.commit()


st.set_page_config(page_title="Classroom Monitoring", layout="centered")
st.title(" Classroom Monitoring System")
st.write("Classificação do comportamento do aluno em sala de aula")

uploaded_file = st.file_uploader("Envie uma imagem", type=["jpg", "png", "jpeg"])

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Imagem enviada", use_column_width=True)

    img = image.resize(IMG_SIZE)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)
    idx = np.argmax(preds)
    confidence = float(np.max(preds))
    label = CLASSES[idx]

    st.subheader(" Resultado")
    st.write(f"**Classe:** {label}")
    st.write(f"**Confiança:** {confidence:.2%}")

    save_interaction(label, confidence)
    st.success("Interação salva no banco de dados!")

st.markdown("---")
st.subheader(" Histórico de Interações")

if st.button("Mostrar histórico"):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM interactions ORDER BY id DESC")
    rows = cursor.fetchall()

    if rows:
        for r in rows:
            st.write(f" {r[1]} |  {r[2]} |  {r[3]:.2%}")
    else:
        st.write("Nenhuma interação registrada.")


In [ ]:
!streamlit run app.py &>/content/logs.txt &


In [ ]:
!cloudflared tunnel --url http://localhost:8501
